In [36]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import shutil

import time
import pandas as pd
from tqdm import tqdm

from itertools import compress

import re

In [37]:
# 0. 환경설정 및 세팅
try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
except FileNotFoundError:
    pass

PATH = r"C:\Users\NH\Documents\chromedriver-win64\chromedriver-win64\chromedriver.exe"
driver=webdriver.Chrome(PATH)

C:\Users\NH\AppData\Local\Temp\ipykernel_6084\78490518.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)


In [38]:
# 1. 날짜와 마지막 페이지 지정
start_day = '2023-09-20'
max_page = 3000 # 이거 설정하지 않아도 되는 법 고민 

In [39]:
def elementToTextList(class_name):
    results = []
    elements = driver.find_elements(By.CLASS_NAME,class_name)
    for element in elements:
        element = element.text.split('\n')
        results.append(element)
    return results

In [40]:
def pagePaser():
    mul_list_txt_box = elementToTextList("mul-list-txt-box")
    mul_list_img_box = elementToTextList("mul-list-img-box")
    mul_list = [mul_list_txt_box,mul_list_img_box]
    return mul_list

In [41]:
df_list = []
for page in tqdm(range(0,max_page*60,60)):
    target_url = 'https://madangs.com/search?addr=&court=&asset_classification=7+10+8+6+1+2+5+3&build_area_max=0&build_area_min=0&disposal_method=0&eval_p_max=0&eval_p_min=0&g_use_type=&land_area_max=0&land_area_min=0&list_type=0&low_p_max=0&low_p_min=0&state=10&g_state=10+20&share=2&g_share=2&special=&contain_special=0&uchal=&use_type=&sort=bd_desc&start_date={}&end_date=&page={}'.format(start_day, page)
    driver.get(target_url)
    df = pagePaser()
    cnt = 0
    while len(df[0]) == 0:
        df = pagePaser()
        cnt += 1
    if cnt > 300:
        break
    df_list.append(df)

 12%|█████████▍                                                                   | 366/3000 [22:34<2:42:28,  3.70s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF64A9678A2+54818]
	(No symbol) [0x00007FF64A8D6AD2]
	(No symbol) [0x00007FF64A78DA3B]
	(No symbol) [0x00007FF64A76DEE8]
	(No symbol) [0x00007FF64A7F3237]
	(No symbol) [0x00007FF64A806C6F]
	(No symbol) [0x00007FF64A7EE883]
	(No symbol) [0x00007FF64A7C3691]
	(No symbol) [0x00007FF64A7C48D4]
	GetHandleVerifier [0x00007FF64ACCB9A2+3610402]
	GetHandleVerifier [0x00007FF64AD21870+3962352]
	GetHandleVerifier [0x00007FF64AD19D5F+3930847]
	GetHandleVerifier [0x00007FF64AA03656+693206]
	(No symbol) [0x00007FF64A8E1638]
	(No symbol) [0x00007FF64A8DD944]
	(No symbol) [0x00007FF64A8DDA72]
	(No symbol) [0x00007FF64A8CE123]
	BaseThreadInitThunk [0x00007FF8550C257D+29]
	RtlUserThreadStart [0x00007FF856B2AA68+40]


In [42]:
def datapre(mul_list_txt_box,mul_list_img_box):
    tags_list = []
    rent_list = []
    for x in mul_list_txt_box:
        idx = ['#' in y for y in x ]
        tags = list(compress(x, idx))
        [x.remove(t) for t in tags]
        tags = ''.join(tags)
        tags_list.append(tags)
    
    for x in mul_list_txt_box:
        idx = ['%)' in y for y in x ]
        tags = list(compress(x, idx))
        [x.remove(t) for t in tags]
        
    for x in mul_list_txt_box:
        if '미정' in x:
            x.remove('미정')
    
    for x in mul_list_txt_box:
        if '미정' in x:
            x.remove('미정')
            
    for x in mul_list_img_box:
        if '임대' in x:
            x.remove('임대')
            rent_list.append(True)
        else:
            rent_list.append(False)         

    for x in mul_list_txt_box:
        if '진행중' in x:
            x.reverse()
            x.append('미정')
            x.reverse()

    for x in mul_list_txt_box:
        if '준비중' in x:
            x.reverse()
            x.append('미정')
            x.reverse()
            
    for x in mul_list_txt_box:
        if '입찰마감' in x[0]:
            x.reverse()
            x.append('미정')
            x.reverse()
            
    df = pd.DataFrame(mul_list_txt_box,columns = ['법원명','사건번호','매각기일','주소','감정가','최저가'])
    df2 = pd.DataFrame(mul_list_img_box,columns = ['주거형태','건물평수','조회수','경매/공매'])
    df['태그'] = tags_list
    df2['임대여부'] = rent_list
    df = pd.concat([df,df2], axis = 1)
    return df

In [43]:
dfs = []
for idx, _ in enumerate(df_list):
    df = datapre(df_list[idx][0],df_list[idx][1])
    dfs.append(df)

In [44]:
dfs = pd.concat(dfs)

In [45]:
dfs = dfs.drop_duplicates().reset_index(drop = True)

In [46]:
dfs = dfs.dropna()

In [47]:
def korean_to_int(korean):
    parts = re.split("(억|만|원)", korean)
    tmp = []
    for x in parts:
        while len(x) != 4:
            x = '0' + x
        tmp.append(x)

    tmp = tmp[:-1]

    tmp = [x for x in tmp if '억' not in x]
    tmp = [x for x in tmp if '만' not in x]
    tmp = [x for x in tmp if '원' not in x]

    result = int(''.join(tmp))
    return result

In [48]:
tmp = []
for x in dfs['감정가']:
    if x == '-':
        result = 0
        tmp.append(result)
    else:
        result = korean_to_int(x)
        tmp.append(result)

In [49]:
dfs['감정가'] = tmp

In [50]:
tmp = []
for x in dfs['최저가']:
    if x == '-':
        result = 0
        tmp.append(result)
    elif x == '비공개':
        result = 0
        tmp.append(result)
    else:
        result = korean_to_int(x)
        tmp.append(result)

In [51]:
dfs['최저가'] = tmp

In [52]:
tmp = []
for x in dfs['조회수']:
    if 'k' in x:
        x = x.replace('view','')
        x = x.replace('k','')
        x = int(float(x)*1000)
        tmp.append(x)
    else:
        x = int(x.replace('view',''))
        tmp.append(x)

In [53]:
dfs['조회수'] = tmp

In [54]:
dfs['건물평수'] = [x.replace('평','') for x in dfs['건물평수']]

In [55]:
dfs['할인율'] = round((dfs['감정가'] - dfs['최저가']) / dfs['감정가'],2)

In [56]:
dfs.to_csv('data.csv',index = False)

In [57]:
dfs

,법원명,사건번호,매각기일,주소,감정가,최저가,태그,주거형태,건물평수,조회수,경매/공매,임대여부,할인율
0,미정,준비중,시작 2023.12.18 10:00 ~,경기도 수원시 영통구 이의동 1256 제지1층 제101호 근린생활시설,942000000,686718000,#기타일반재산#유찰3회,근린생활시설,27,648,공매,0.0,0.27
1,미정,진행중,~ 2023.11.29 16:00 마감,화성 전곡해양 일반산업단지 지원 5-1용지 (전곡리 1084-1),673534500,673534500,#기타일반재산#유찰2회,대지,187,1500,공매,0.0,0.00
2,미정,진행중,~ 2023.11.29 16:00 마감,화성 전곡해양 일반산업단지 지원 5-3용지 (전곡리 1084-3),445067000,445067000,#기타일반재산#유찰2회,대지,125,672,공매,0.0,0.00
3,미정,진행중,~ 2023.11.29 16:00 마감,화성 전곡해양 일반산업단지 지원 9-1용지 (전곡리 1113-1),748800000,748800000,#기타일반재산#유찰2회,대지,218,410,공매,0.0,0.00
4,미정,진행중,~ 2023.11.29 16:00 마감,화성 전곡해양 일반산업단지 지원 9-2용지 (전곡리 1113-2),776880000,776880000,#기타일반재산#유찰2회,대지,226,282,공매,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20298,미정,인터넷입찰마감,개찰 2023.09.21 09:00,인천광역시 미추홀구 주안동 94-3 올레오아파트 207호 307호 1002호 120...,1168000000,830000000,#기타일반재산#유찰11회,기타주거용건물,79,43,공매,0.0,0.29
20299,서산지원,2022타경1867[1],결과대기,"충청남도 당진시 동부로 99, 102동 10층1003호 (수청동,당진수청한라비발디캠퍼스)",278000000,278000000,#유찰1회#대지권미등기,아파트,19,318,경매,0.0,0.00
20300,수원지법,2022타경68192[1],결과대기,경기도 용인시 처인구 양지면 대대리 산12-10,33100000,11353000,#유찰3회#지분매각,임야,201,142,경매,0.0,0.66
20301,인천지법,2022타경9295[1],결과대기,"인천광역시 서구 크리스탈로 148, 376동 18층1803호 (경서동,청라한화꿈에그린)",984000000,984000000,#신건,아파트,31,77,경매,0.0,0.00
